<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=79dcdbc193c03a7a53017ac284b7c77a71c2fe020d36c9520db7f4288e1399a8
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-07 11:18:07
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.42 C
-------------------
Today PnL: -88.66 K (-0.62%)
Current PnL: -22.51 L (-14.75%)
CY Booked + Current PnL: -9.86 L (-6.46%)
-------------------
Total profit:  1.76 L
Total loss:  -24.27 L
-------------------
Total Booked + Current PnL: 17.12 L (13.63%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.91%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 89.50 L (63.0%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.19%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-6.91,67.0,X-LC,6.48,233189.0,22823.0,5970.0,-0.09,10.85,2.56,13.68,37.0,3.82,1.65,29.16,XY25,NTT,REFINERIES
78,TTKPRESTIG,770.00,102.56,53.0,M-SC,1.93,87298.0,-13479.0,13575.0,0.46,-13.38,15.55,0.09,245.0,-0.99,0.62,14.69,OX40N,NTT,DURABLES
79,UNIONBANK,163.00,-5.43,65.0,M-LC,11.74,171573.0,30733.0,13760.0,0.50,21.82,8.02,31.59,66.0,2.23,1.22,54.09,XY24,NTT,BANKS
2,ABBOTINDIA,35195.00,-11.06,39.0,X-MC,2.83,87930.0,-2748.0,17656.0,-0.49,-3.03,20.08,16.44,101.0,-0.16,0.62,16.53,X40,ATH,PHARMA
33,ICICIGI,2252.93,-14.77,53.0,X-MC,5.77,146764.0,10631.0,19960.0,-0.88,7.81,13.60,22.47,91.0,0.53,1.04,21.80,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SFL,1287.0,35.77,52.0,M-SC,3.53,179258.0,-82960.0,152782.0,3.47,-31.64,85.23,26.63,229.0,-0.54,1.27,11.45,XY24,NTT,MISC
82,VALIANTORG,838.0,-285.28,43.0,H-SC,11.27,102969.0,-60636.0,181946.0,1.61,-37.06,176.70,74.15,139.0,-0.33,0.73,31.99,XR,NTT,CHEMICALS
37,INDIGOPNTS,1408.0,89.73,37.0,M-SC,6.37,124533.0,-50026.0,50062.0,1.36,-28.66,40.20,0.02,221.0,-1.00,0.88,8.32,OX40N,NTT,PAINTS
84,VOLTAS,1530.0,-6.39,33.0,X-MC,4.35,196335.0,4593.0,33161.0,1.28,2.40,16.89,19.69,99.0,0.14,1.39,9.36,XY25,NTT,AC
35,IEX,219.0,-33.63,47.0,H-SC,16.00,189668.0,-10286.0,107523.0,1.18,-5.14,56.69,48.63,115.0,-0.10,1.35,5.81,XR,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SURYODAY,240.0,55.70,42.0,H-SC,15.70,142800.0,-36271.0,102002.0,-6.13,-20.26,71.43,36.71,135.0,-0.36,1.01,41.69,XR,NTT,BANKS
1,ABB,7934.0,-42.88,33.0,H-LC,5.13,238824.0,-22795.0,142005.0,-4.83,-8.71,59.46,45.57,7.0,-0.16,1.69,1.62,AR,NTT,ELECTRICAL
71,SYMPHONY,1306.0,-44.94,42.0,M-SC,16.74,117238.0,-53903.0,53847.0,-4.67,-31.50,45.93,-0.03,196.0,-1.00,0.83,2.21,OX40N,NTT,DURABLES
85,WHIRLPOOL,2270.0,-41.93,47.0,M-SC,6.10,120318.0,5520.0,88518.0,-3.14,4.81,73.57,81.92,223.0,0.06,0.85,42.44,XR,NTT,DURABLES
17,CERA,9475.0,-24.34,42.0,H-SC,1.84,137977.0,-37926.0,79944.0,-3.00,-21.56,57.94,23.89,149.0,-0.47,0.98,19.16,OX40N,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,25.32,45.0,M-LC,9.75,204254.0,-822.0,113381.0,0.5,-0.4,55.51,54.89,52.0,-0.01,1.45,23.6,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,102.56,53.0,M-SC,1.93,87298.0,-13479.0,13575.0,0.46,-13.38,15.55,0.09,245.0,-0.99,0.62,14.69,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,45.0,H-SC,2.76,221724.0,-47943.0,84277.0,-0.73,-17.78,38.01,13.47,138.0,-0.57,1.57,12.74,XY24,NTT,PAINTS
17,CERA,9475.0,-24.34,42.0,H-SC,1.84,137977.0,-37926.0,79944.0,-3.00,-21.56,57.94,23.89,149.0,-0.47,0.98,19.16,OX40N,NTT,CERAMICS
42,JCHAC,2282.0,19921.74,62.0,M-SC,1.51,97860.0,-29945.0,29935.0,-0.03,-23.43,30.59,-0.01,232.0,-1.00,0.69,10.81,OX40N,NTT,AC
67,SIS,528.0,1995.34,42.0,H-SC,2.52,84558.0,-26474.0,50084.0,-2.30,-23.84,59.23,21.26,156.0,-0.53,0.60,14.25,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-5.43,65.0,M-LC,11.74,171573.0,30733.0,13760.0,0.5,21.82,8.02,31.59,66.0,2.23,1.22,54.09,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,49.96,58.0,M-MC,10.26,236344.0,11382.0,156271.0,-0.48,5.06,66.12,74.53,192.0,0.07,1.68,38.95,XY24,BTT,STEEL
46,KPIGREEN,731.05,7.87,56.0,H-SC,8.06,128659.0,3362.0,55568.0,-0.04,2.68,43.19,47.03,141.0,0.06,0.91,61.09,MH,ATH,POWER
79,UNIONBANK,163.00,-5.43,65.0,M-LC,11.74,171573.0,30733.0,13760.0,0.50,21.82,8.02,31.59,66.0,2.23,1.22,54.09,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-41.93,47.0,M-SC,6.10,120318.0,5520.0,88518.0,-3.14,4.81,73.57,81.92,223.0,0.06,0.85,42.44,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.71,50.0,H-SC,5.39,129087.0,5751.0,125873.0,-1.52,4.66,97.51,106.72,119.0,0.05,0.92,29.13,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,7.87,56.0,H-SC,8.06,128659.0,3362.0,55568.0,-0.04,2.68,43.19,47.03,141.0,0.06,0.91,61.09,MH,ATH,POWER
85,WHIRLPOOL,2270.00,-41.93,47.0,M-SC,6.10,120318.0,5520.0,88518.0,-3.14,4.81,73.57,81.92,223.0,0.06,0.85,42.44,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.71,50.0,H-SC,5.39,129087.0,5751.0,125873.0,-1.52,4.66,97.51,106.72,119.0,0.05,0.92,29.13,AR,ATH,MISC
31,HINDZINC,730.22,25.32,45.0,M-LC,9.75,204254.0,-822.0,113381.0,0.50,-0.40,55.51,54.89,52.0,-0.01,1.45,23.60,X5K,ATH,METALS
86,WIPRO,420.00,-16.52,39.0,M-LC,5.41,147033.0,-3912.0,113362.0,-1.21,-2.59,77.10,72.51,53.0,-0.03,1.04,3.27,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,22.0,X-LC,32.34,148050.0,-128473.0,238553.0,0.33,-46.46,161.13,39.81,54.0,-0.54,1.05,4.35,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.87,24.0,X-SC,7.79,86934.0,-41736.0,87030.0,-0.31,-32.44,100.11,35.20,219.0,-0.48,0.62,0.00,X40,NTT,FOOTWEAR
23,DMART,5391.45,-21.48,24.0,X-LC,7.54,200080.0,-10327.0,69488.0,-1.97,-4.91,34.73,28.12,38.0,-0.15,1.42,17.56,X40N,ATH,FMCG
44,JSWINFRA,342.00,-23.75,27.0,X-MC,4.56,191362.0,-8985.0,39478.0,-0.14,-4.48,20.63,15.23,178.0,-0.23,1.36,26.32,X40N,NTT,REALTY
52,PGHH,17907.65,-32.27,30.0,X-MC,0.15,196110.0,-4710.0,72502.0,-0.67,-2.35,36.97,33.76,80.0,-0.06,1.39,2.73,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-32.27,30.0,X-MC,0.15,196110.0,-4710.0,72502.0,-0.67,-2.35,36.97,33.76,80.0,-0.06,1.39,2.73,X40,ATH,FMCG
41,ITC,452.00,-38.34,45.0,X-LC,1.06,198901.0,-1237.0,22118.0,-0.18,-0.62,11.12,10.44,4.0,-0.06,1.41,4.91,X40,NTT,FMCG
66,SIEMENS,4671.50,-5.39,38.0,H-LC,1.39,152170.0,-33925.0,81411.0,-1.30,-18.23,53.50,25.51,15.0,-0.42,1.08,12.14,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-9.30,33.0,X-MC,3.24,302463.0,-25528.0,132055.0,-0.14,-7.78,43.66,32.48,92.0,-0.19,2.15,2.75,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-6.38,64.0,X-LC,3.96,245513.0,11369.0,37244.0,0.77,4.86,15.17,20.77,86.0,0.31,1.74,12.11,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,24.0,X-SC,7.79,86934.0,-41736.0,87030.0,-0.31,-32.44,100.11,35.20,219.0,-0.48,0.62,0.00,X40,NTT,FOOTWEAR
53,QUESS,986.00,-50.41,33.0,X-SC,37.43,49054.0,-15952.0,166882.0,0.39,-24.54,340.20,232.18,198.0,-0.10,0.35,0.39,XY24,NTT,MISC
51,PAGEIND,51769.92,-29.72,32.0,X-MC,7.41,79400.0,-4110.0,24138.0,-0.15,-4.92,30.40,23.98,82.0,-0.17,0.56,0.61,X40,ATH,APPARELS
20,COLPAL,3726.84,-7.40,39.0,X-MC,7.30,216020.0,-47345.0,156658.0,-0.62,-17.98,72.52,41.51,84.0,-0.30,1.53,1.37,XY25,ATH,FMCG
1,ABB,7934.00,-42.88,33.0,H-LC,5.13,238824.0,-22795.0,142005.0,-4.83,-8.71,59.46,45.57,7.0,-0.16,1.69,1.62,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.41,33.0,X-SC,37.43,49054.0,-15952.0,166882.0,0.39,-24.54,340.20,232.18,198.0,-0.10,0.35,0.39,XY24,NTT,MISC
51,PAGEIND,51769.92,-29.72,32.0,X-MC,7.41,79400.0,-4110.0,24138.0,-0.15,-4.92,30.40,23.98,82.0,-0.17,0.56,0.61,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.56,41.0,X-SC,4.22,81019.0,-64141.0,140074.0,-1.47,-44.19,172.89,52.31,136.0,-0.46,0.57,8.45,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-11.06,39.0,X-MC,2.83,87930.0,-2748.0,17656.0,-0.49,-3.03,20.08,16.44,101.0,-0.16,0.62,16.53,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-41.87,24.0,X-SC,7.79,86934.0,-41736.0,87030.0,-0.31,-32.44,100.11,35.20,219.0,-0.48,0.62,0.00,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.96,-29.65,44.0,X-LC,9.73,279791.0,-66165.0,132873.0,-1.14,-19.13,47.49,19.28,1.0,-0.50,1.98,3.43,X40,ATH,IT
39,INFY,2275.00,-19.96,47.0,X-LC,4.99,311534.0,-1354.0,173026.0,-0.28,-0.43,55.54,54.87,3.0,-0.01,2.21,7.81,X40,BTT,IT
41,ITC,452.00,-38.34,45.0,X-LC,1.06,198901.0,-1237.0,22118.0,-0.18,-0.62,11.12,10.44,4.0,-0.06,1.41,4.91,X40,NTT,FMCG
83,VBL,671.64,-17.03,52.0,X-LC,6.35,298648.0,-17194.0,129852.0,-0.79,-5.44,43.48,35.67,5.0,-0.13,2.12,7.66,X40N,ATH,FMCG
1,ABB,7934.00,-42.88,33.0,H-LC,5.13,238824.0,-22795.0,142005.0,-4.83,-8.71,59.46,45.57,7.0,-0.16,1.69,1.62,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,32.0,L-SC,27.47,73152.0,-40397.0,80445.0,-2.83,-35.58,109.97,35.27,268.0,-0.50,0.52,79.04,XR,NTT,HOTELS
7,ASIANTILES,137.00,7344.44,62.0,L-SC,12.29,81669.0,-12141.0,88758.0,-2.10,-12.94,108.68,81.67,269.0,-0.14,0.58,58.15,XR,NTT,CERAMICS
49,MASFIN,398.61,-19.15,50.0,H-SC,8.35,92100.0,-5880.0,27483.0,0.92,-6.00,29.84,22.05,152.0,-0.21,0.65,33.87,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-41.93,47.0,M-SC,6.10,120318.0,5520.0,88518.0,-3.14,4.81,73.57,81.92,223.0,0.06,0.85,42.44,XR,NTT,DURABLES
46,KPIGREEN,731.05,7.87,56.0,H-SC,8.06,128659.0,3362.0,55568.0,-0.04,2.68,43.19,47.03,141.0,0.06,0.91,61.09,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,73.00,64.0,H-SC,3.99,150137.0,-32638.0,143711.0,0.09,-17.86,95.72,60.77,125.0,-0.23,1.06,35.21,XR,NTT,JEWELLERY
46,KPIGREEN,731.05,7.87,56.0,H-SC,8.06,128659.0,3362.0,55568.0,-0.04,2.68,43.19,47.03,141.0,0.06,0.91,61.09,MH,ATH,POWER
68,SONACOMS,806.63,-32.06,66.0,M-SC,6.46,84569.0,-16590.0,56594.0,0.07,-16.40,66.92,39.54,202.0,-0.29,0.60,19.25,AR,ATH,AUTO
7,ASIANTILES,137.00,7344.44,62.0,L-SC,12.29,81669.0,-12141.0,88758.0,-2.10,-12.94,108.68,81.67,269.0,-0.14,0.58,58.15,XR,NTT,CERAMICS
62,SAMMAANCAP,326.00,-196.59,68.0,M-SC,33.96,166482.0,17262.0,126926.0,-0.17,11.57,76.24,96.62,208.0,0.14,1.18,80.17,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.71
1,25,44.32
2,50,75.46


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.75
LC    33.16
MC    23.03
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.00
X40      16.24
X40N     12.43
XY25     11.96
XR        9.83
AR        7.78
OX40N     6.52
X200      1.78
X5K       1.45
SR        1.04
MH        0.91
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.24
X-LC    22.50
X-MC    18.71
M-SC    12.59
X-SC     5.49
M-LC     5.13
H-LC     4.55
H-MC     2.32
M-MC     1.68
L-SC     1.43
L-LC     0.98
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.29,-4.90,38.81
IT,12.43,-19.82,84.01
FINANCE,12.16,-10.21,60.16
MISC,7.79,-18.24,77.89
ELECTRICAL,5.69,-12.69,52.64
PAINTS,5.60,-16.76,34.17
BANKS,4.01,-19.55,89.10
INSURANCE,3.86,-1.56,39.43
AUTO,2.72,-50.07,112.70


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3345649.0,22
XR,1268487.0,13
AR,1189713.0,9
X40,865379.0,12
X40N,715120.0,9
OX40N,564146.0,9
XY25,484646.0,8
SR,274755.0,2
X5K,113381.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3412952.0,24
M-SC,1267953.0,15
X-MC,1233728.0,14
X-LC,1104212.0,13
X-SC,574896.0,6
H-LC,296256.0,3
M-LC,285213.0,4
H-MC,260456.0,2
L-SC,259415.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1199850.0      6
           AR         842215.0      5
           XR         790410.0      7
M-SC       XY24       731631.0      6
X-MC       XY24       577329.0      4
X-LC       X40        469321.0      5
X-SC       X40N       320984.0      4
X-LC       XY24       315831.0      2
X-MC       X40        309028.0      6
H-SC       SR         274755.0      2
           OX40N      250154.0      3
X-LC       X40N       236584.0      3
H-LC       AR         223416.0      2
X-MC       XY25       189819.0      2
H-MC       XY24       184095.0      1
L-SC       XR         169203.0      2
X-SC       XY24       166882.0      1
X-MC       X40N       157552.0      2
M-MC       XY24       156271.0      1
M-SC       OX40N      147419.0      4
           XR         137895.0      2
           XY25       126926.0      1
           AR         124082.0      2
M-LC       X5K        113381.0      1
           XR         113362.0      1
L-SC       OX40N       90212.0      1
X-SC       X40         87030.0      1
X-LC       XY25        82476.0      3
H-MC       OX40N       76361.0      1
H-LC       X200        72840.0      1
L-MC       XR          57617.0      1
H-SC       MH          55568.0      1
M-LC       XY25        44710.0      1
L-LC       XY25        40715.0      1
M-LC       XY24        13760.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
